# API Tutorial - Fetching Horse Racing Market Data

## Overview
This tutorial will walk you through the process of connecting to Betfair's Stream API, streaming data and placing bets in python. It will utilise the [betfairlightweight](https://github.com/liampauling/betfair) python library.

## Disclaimer
It should be noted that this guide is in no way comprehensive. There are many aspects that are missing which are fully detailed in the [official documentation](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-Overview). Please read this <b>thoroughly</b> and follow best practice. Betfair Australia is in no way liable for your losses.

### Requirements
* This tutorial will assume that you have an API key. If you don't, please follow the steps outlined on [The Automation Hub](https://betfair-datascientists.github.io/api/apiappkey/).
* This tutorial will also assume that you have a stream API app key.

### Quick Links
Other useful links are outlined below.
* [Developer Docs](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-Overview) - The official Dev Docs for the Stream API
* [Sports API Visualiser](https://docs.developer.betfair.com/visualisers/api-ng-sports-operations/) - Useful for exploring what the API has to offer
* [Account API Visualiser](https://docs.developer.betfair.com/visualisers/api-ng-account-operations/)
* [Examples using betfairlightweight](https://github.com/liampauling/betfair/tree/master/examples)

## Getting Started
### Installing betfairlightweight
We need to install betfairlightweight. To do this, simply use `pip install betfairlightweight` in the cmd prompt/terminal. If this doesn't work, you will have to google your error. If you're just starting out with python, you may have to add python to your environment variables.

### Setting Up Your Certificates
To use the API securely, Betfair recommends generating certificates. The betfairlight package requires this to login non-interactively. For detailed instructions on how to generate certificates on a windows machine, follow the instructions outlined [here](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Certificate+Generation+With+XCA). For alternate instructions for windows, or for Mac/Linux machines, follow the instructions outlined [here](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Non-Interactive+%28bot%29+login). You should then create a folder for your certs, perhaps named 'certs' and grab the path loation.

If you can't set up your certs, you can still use the stream API; however it is less secure. See below for details.

## Getting Stream Data
### Log into the API Client
Now we're finally ready to log in and use the API. First, we create an APIClient object and then log in.

You'll also need to change the username, password and app_key variables to your own.

In [ ]:
# Import libraries
import os
import betfairlightweight
import pandas as pd
import queue
import datetime
from IPython.display import Image

# Change this certs path to wherever you're storing your certificates
certs_path = "your_certs_path"

# Change these login details to your own
username = "your_username"
pw = "your_password"
app_key = "your_app_key"

# Log in - if you have set up your certs use this
trading = betfairlightweight.APIClient(username, pw, app_key=app_key, certs=certs_path)
trading.login()

# Log in - if you haven't set up your certs use this
# trading = betfairlightweight.APIClient(username, pw, app_key=app_key)
# trading.login_interactive()

### Create a Queue, Listener and Stream
Next, we need to create a queue object. The idea of a queue is basically like it sounds; data is put into a queue and you can consume/read the data at the front of the queue.

The listener object we are creating simply 'listens' to the data from the API as it changes.

In [2]:
# create queue
output_queue = queue.Queue()

# create stream listener
listener = betfairlightweight.StreamListener(
    output_queue=output_queue,
)

# create stream
stream = trading.streaming.create_stream(
    listener=listener,
)

### Find Markets to Stream
Now we need to find specific markets to stream. As thoroughbred markets are volatile, let's look at thoroughbred markets. First, we need to find upcoming thoroughbred markets. To do this we will copy the template we used in our other regular API tutorial and get a DataFrame of upcoming Event IDs.

In [3]:
# Define a market filter
thoroughbreds_event_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'], # Thoroughbreds event type id is 7
    market_countries=['AU'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime("%Y-%m-%dT%TZ")
    }
)

# Get a list of all thoroughbred events as objects
aus_thoroughbred_events = trading.betting.list_events(
    filter=thoroughbreds_event_filter
)

# Create a DataFrame with all the events by iterating over each event object
aus_thoroughbred_events_today = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in aus_thoroughbred_events],
    'Event ID': [event_object.event.id for event_object in aus_thoroughbred_events],
    'Event Venue': [event_object.event.venue for event_object in aus_thoroughbred_events],
    'Country Code': [event_object.event.country_code for event_object in aus_thoroughbred_events],
    'Time Zone': [event_object.event.time_zone for event_object in aus_thoroughbred_events],
    'Open Date': [event_object.event.open_date for event_object in aus_thoroughbred_events],
    'Market Count': [event_object.market_count for event_object in aus_thoroughbred_events],
    'Local Open Date': [event_object.event.open_date.replace(tzinfo=datetime.timezone.utc).astimezone(tz=None) 
                        for event_object in aus_thoroughbred_events]
})

aus_thoroughbred_events_today

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count,Local Open Date
0,Bath (AUS) 31st Oct,28984058,Bathurst,AU,Australia/Sydney,2018-10-31 07:15:00,16,2018-10-31 18:15:00+11:00
1,Rand (AUS) 31st Oct,28983416,Randwick,AU,Australia/Sydney,2018-10-31 03:00:00,4,2018-10-31 14:00:00+11:00
2,Gosf (AUS) 1st Nov,28985400,Gosford,AU,Australia/Sydney,2018-11-01 02:35:00,12,2018-11-01 13:35:00+11:00
3,Crns (AUS) 31st Oct,28983577,Cairns,AU,Australia/Queensland,2018-10-31 02:19:00,4,2018-10-31 13:19:00+11:00
4,Asct (AUS) 31st Oct,28983570,Ascot,AU,Australia/Perth,2018-10-31 05:34:00,12,2018-10-31 16:34:00+11:00
5,Bend (AUS) 31st Oct,28982866,Bendigo,AU,Australia/Sydney,2018-10-31 01:00:00,2,2018-10-31 12:00:00+11:00
6,Mdra (AUS) 31st Oct,28983440,Mildura,AU,Australia/Sydney,2018-10-31 07:27:00,16,2018-10-31 18:27:00+11:00
7,Redc (AUS) 31st Oct,28983189,Redcliffe,AU,Australia/Queensland,2018-10-31 07:53:00,18,2018-10-31 18:53:00+11:00
8,Geel (AUS) 1st Nov,28985610,Geelong,AU,Australia/Sydney,2018-11-01 02:30:00,14,2018-11-01 13:30:00+11:00
9,Sapp (AUS) 1st Nov,28985640,Sapphire Coast,AU,Australia/Sydney,2018-11-01 02:50:00,12,2018-11-01 13:50:00+11:00


Now that we have event IDs, we need to get markets within the events. Let's take the first event ID from our DataFrame and find the markets for that event. We will then choose the first event to run.

In [15]:
first_event_id = aus_thoroughbred_events_today.iloc[0]['Event ID']

market_catalogue_filter = betfairlightweight.filters.market_filter(event_ids=[first_event_id])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START',
)

# Create a DataFrame for each market catalogue
markets = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
})

# Get the first race
markets[markets['Market Name'].str.contains('R')]

,Market Name,Market ID,Total Matched
0,R1 1550m 3yo,1.150429726,0.0
1,R2 1100m 3yo,1.150429728,0.0
3,R3 1800m Hcap,1.150429730,0.0
5,R4 1550m Hcap,1.150429733,0.0
7,R5 1100m Hcap,1.150429735,0.0
9,R6 1300m Hcap,1.150429737,0.0
11,R7 1100m Hcap,1.150429739,0.0


In [16]:
# Get the market id
market_id = markets.loc[markets['Market Name'].str.contains('R1'), 'Market ID'].values[0]

### Subscribing to Markets
To tell the listener which markets we want to view data for, we need to subscribe to a market(s). First we will define filters, then we will subscribe to the market.

In [17]:
# Create Filters
market_filter = betfairlightweight.filters.streaming_market_filter(
    event_type_ids=['7'],
    country_codes=['AU'],
    market_types=['WIN'],
    market_ids=[market_id] # Add the market ID we found above to the filter.
)

market_data_filter = betfairlightweight.filters.streaming_market_data_filter(
    fields=['EX_BEST_OFFERS', 'EX_MARKET_DEF'],
    ladder_levels=3,
)

# Subscribe
streaming_unique_id = stream.subscribe_to_markets(
    market_filter=market_filter,
    market_data_filter=market_data_filter,
    conflate_ms=1000,  # send update every 1000ms
    initial_clk=listener.initial_clk, # These allow for recovery if you disconnect
    clk=listener.clk
)

### Getting a Snap of a Market
If we want to see a quick snapshot of the market, we can use the `snap` method from the `StreamListener` class. This will give us a snap of the market which is a great way of getting a quick view of the market. We will use a function we have defined in the previous tutorial to parse the data.

In [109]:
# Start stream
stream.start(_async=True)

market_books = stream.listener.snap(
    market_ids=[market_id]
)

stream.stop()


def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 0.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 0.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df


# Process snap
processing_functions.process_runner_books(market_books[0].runners)

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,20575661,1.03,15.25,1000.0,1.01,None,None,ACTIVE,None,44.265
1,21396183,1.03,15.25,1000.0,1.01,None,None,ACTIVE,None,11.809
2,21396182,1.03,15.25,1000.0,1.01,None,None,ACTIVE,None,26.288
3,3241355,1.03,15.25,1000.0,1.01,None,None,ACTIVE,None,17.639


### Get Stream Updates
Now we can get stream updates. Note that each stream has a unique ID that you can use to differentiate data if you were subscribed to multiple markets.

The first object received will be an 'image' and will contain the key 'img' with the value True, to indicate that it is an image. An image is basically a snapshot of the market at that time, so that you can simply receive updates and then update the image. An exmaple of an image received is:

```
{'id': '1.150502488', 'marketDefinition': {'bspMarket': True, 'turnInPlayEnabled': True, 'persistenceEnabled': True, 'marketBaseRate': 6, 'eventId': '28985610', 'eventTypeId': '7', 'numberOfWinners': 1, 'bettingType': 'ODDS', 'marketType': 'WIN', 'marketTime': '2018-11-01T04:00:00.000Z', 'suspendTime': '2018-11-01T04:00:00.000Z', 'bspReconciled': False, 'complete': True, 'inPlay': False, 'crossMatching': True, 'runnersVoidable': False, 'numberOfActiveRunners': 5, 'betDelay': 0, 'status': 'OPEN', 'runners': [{'adjustmentFactor': 3.874, 'status': 'ACTIVE', 'sortPriority': 1, 'id': 21411481}, {'adjustmentFactor': 15.384, 'status': 'ACTIVE', 'sortPriority': 2, 'id': 21411482}, {'adjustmentFactor': 31.995, 'status': 'ACTIVE', 'sortPriority': 3, 'id': 21411483}, {'adjustmentFactor': 36.709, 'status': 'ACTIVE', 'sortPriority': 4, 'id': 21411484}, {'adjustmentFactor': 12.039, 'status': 'ACTIVE', 'sortPriority': 5, 'id': 17312089}], 'regulators': ['MR_NJ', 'MR_INT'], 'venue': 'Geelong', 'countryCode': 'AU', 'discountAllowed': True, 'timezone': 'Australia/Sydney', 'openDate': '2018-11-01T02:30:00.000Z', 'version': 2488592392, 'raceType': 'Flat', 'priceLadderDefinition': {'type': 'CLASSIC'}}, 'rc': [{'batb': [[0, 23, 14.4], [1, 22, 2.46], [2, 21, 2]], 'batl': [[0, 24, 2.57], [1, 36, 5.15], [2, 40, 1.38]], 'id': 17312089}, {'batb': [[0, 2.82, 2.77], [1, 2.8, 10], [2, 2.78, 12]], 'batl': [[0, 2.84, 78.02], [1, 2.86, 42.18], [2, 2.88, 11.08]], 'id': 21411484}, {'batb': [[0, 3.65, 256.75], [1, 3.6, 14], [2, 3.55, 12.2]], 'batl': [[0, 3.7, 185.43], [1, 3.75, 55.39], [2, 3.8, 2.82]], 'id': 21411483}, {'batb': [[0, 16.5, 12.17], [1, 16, 2.29], [2, 15.5, 4]], 'batl': [[0, 17, 50.12], [1, 17.5, 2.5], [2, 18, 22.16]], 'id': 21411482}, {'batb': [[0, 3.6, 209.55], [1, 3.55, 292.28], [2, 3.5, 59]], 'batl': [[0, 3.65, 65.88], [1, 3.7, 18], [2, 3.75, 150.01]], 'id': 21411481}], 'img': True}

```

After this you will get updates to this image. For example, you may get an update like the following:
```
{'id': '1.150366126', 'rc': [{'batb': [[2, 1.06, 8.01], [1, 1.11, 55.32], [0, 1.12, 48.21]], 'batl': [[1, 1.53, 22.13], [0, 1.46, 13.83]], 'id': 21336569}], 'con': True}
```

This just means that there have been runner changes (rc) for the runner with a runner ID of 21336569. See the below section for how to interpret the changes.

<b>We will not run the stream in this Notebook as it will most likely cause it to crash. However, run the stream_api_tutorial.py script if you wish to see the stream in action.</b>

Instead, I will copy and paste an example of what the outlook should look like.

In [ ]:
# while True:
#     # Get the market books at the front of the queue
#     print("Getting market books")
#     market_books = output_queue.get()
#     print(market_books)
#     # Iterate over the market books and print updates, times etc.
#     for market_book in market_books:
#         print("This is the market book object:", market_book, "\n")
#         print("This is the time of update:", market_book.publish_time, "\n")
#         print("This is the actual streaming updated data:", market_book.streaming_update)

example_output = '''MarketBook
Time of update: 2018-10-30 03:09:55.118000
{'id': '1.150366126', 'rc': [{'batb': [[2, 1.06, 8.01], [1, 1.11, 55.32], [0, 1.12, 48.21]], 'batl': [[1, 1.53, 22.13], [0, 1.46, 13.83]], 'id': 21336569}], 'con': True}
Getting market books'''

print(example_output)

### Interpreting Updates
In the printed strings above, we have first printed a Market Book object. This is what is returned when we get the first item in the queue. From the market book we can extract a heap of information, including the runners and prices.

The second print statement prints the publish time; this is the time that prices changed.

The third print statement prints the actual changes. Here we can see that for the market ID 1.150366126, the Best Available To Back (batb) for selection ID 21336569 is 1.12.

The following image will be used as an example.
![order_book_example](images/order_book_example.png)

This image shows a typical Best Available To Back side of a market on Betfair. If you set `ladder_levels=3` in the market_data_filter, you will get the top 3 Best Available To Backs, like in the example image, whereas if you set `ladder_levels=1` you will only get the top Best Available To Back data (in this case, the price is 1.62 and the size is 54).

Say we set `ladder_levels=3`. In this example, the stream API would return something within the 'rc' key like:
[{'batb': [[0, 1.62, 54], [1, 1.61, 143], [2, 1.6, 332]}]. Here, the 0 refers to the top level of the ladder - i.e. the top Best Available To Back data (price - 1.62, size - 54), the 1 refers to the second level of the ladder - i.e. the second Best Available To Back data (price - 1.61, size - 143) and the 2 refers to the third level of the ladder - i.e. the third Best Available To Back data (price - 1.6, size - 332).

If you want data spanning more ladder levels, simply change the `ladder_levels` argument in the market_data_filter.

### Creating Orders and Recieving Order Updates
To create an order, you simply create it like you would have using the regular API. However, you can receive Order Change messages - similar to the market change messages outlined above. In future versions of this tutorial we will provide an overview of interpreting order change messages; but for now Betfair has fantastic documentation [here.](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-OrderSubscriptionMessage)